In [1]:
%%time
import json
import inspect
import tqdm
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

import Artha
from Artha.analysis import *
from Artha.nlp_extraction import *


CPU times: user 1.66 s, sys: 259 ms, total: 1.92 s
Wall time: 2.22 s


In [2]:
%%time
# user = "naval"
user = "CryptoKaleo"
with open("../data/tweets/u"+user+"tweets.json", "r") as r:
    tweets = json.load(r)
tweet_text = []
for ind, tweet in enumerate(tweets):
    sent = tweet["full_text"]
    
    try: #remove tweets of only tagging someone
        while sent[0] == "@":
            sent = sent.split(" ", 1)[1]
        tweet_text.append(sent.replace("&amp;", "and").replace("@", ""))
    except:
        tweets.pop(ind)
# tweet_text = [tweet["full_text"] for tweet in tweets]

CPU times: user 62.7 ms, sys: 12.3 ms, total: 75 ms
Wall time: 74 ms


In [12]:
%%time
spacy_sents = []
# need to disable pipes
with open("senti2.txt", "w+") as w:
    for ind, doc in enumerate(nlp.pipe(tweet_text, n_process=-1)):
        # tick = [ent.text for ent in doc.ents if ent.label_ in ent_labels]
        # labels = [ent.label_ for ent in doc.ents if ent.label_ in ent_labels]
        tick = doc._.tickers
        for i in tick:
            if i not in tickers.keys() and i not in tickers.values():
                tick.remove(i)
        if tick: # 
            w.write(" ".join(tick)+"\n")
            w.write(str(ind)+" "+doc.text+"\n\n")
            spacy_sents.append(ind)
        
        #     if abs(doc._.polarity)>0:
        #         w.write(" ".join(tick)+"\n")
        #         w.write(" ".join(labels)+"\n")
        #         # w.write("\n")
        #         # w.write(str(doc._.polarity)+" "+str(doc._.subjectivity)+"\n")
        #         # vs = analyzer.polarity_scores(sent)
        #         # w.write("{:-<65} {}\n\n".format(str(ind)+" "+sent, str(vs)))
        #         w.write(str(ind)+" "+doc.text+"\n\n")
        # if not tick:
        #     w.write(str(ind)+" "+doc.text+"\n\n")
            

CPU times: user 1.96 s, sys: 122 ms, total: 2.08 s
Wall time: 4.21 s


In [5]:
%%time
ent_labels = ["CRYPTO", "ORG", "MONEY"]

doc = nlp(tweet_text[28])
print(doc.text, "\n")
tick = [ent.text for ent in doc.ents if ent.label_ in ent_labels]
for i in tick:
    print(i)
    if i not in tickers.keys() and i not in tickers.values():
#         tick.remove(i)
        print("Not in","\n")
    
print("\n")
#     if i not in tickers.keys() or tickers.values():
#         tick.remove(i)
for ent in doc.ents:
    # if ent.label_ == "ORG" or ent.label_ == "CRYPTO":
        # if i in tickers.keys() or tickers.values()
        # print(ent.text, ent.label_)
    print(ent.text, ent.label_)
print("\n", tick)

Bitcoin is $1,000 lower than it was yesterday at this time, and alts are all printing higher highs across the board led by Ethereum.

This is the moment we've been waiting for.

Salt szn is over. Alt szn is here. 

Bitcoin
1,000
Not in 

Ethereum


Bitcoin ORG
1,000 MONEY
yesterday DATE
Ethereum ORG
Salt PERSON

 ['Bitcoin', '1,000', 'Ethereum']
CPU times: user 16.9 ms, sys: 2.34 ms, total: 19.3 ms
Wall time: 18.1 ms


In [3]:
doc = nlp(tweet_text[1634])
# doc_ents = [ent.text for ent in doc.ents if ent.label_ in ent_labels]
print(sorted(doc._.tickers))

for ent in doc.ents:
    print(ent, ent.label_)

['BTC', 'KIN']
KIN ORG
BTC ORG
ATH ORG
5 CARDINAL
1 CARDINAL
